In [2]:
import pandas as pd
import random
import numpy as np
import glob
from collections import Counter
from collections import defaultdict

In [6]:
random.sample(range(10),5)

[2, 7, 1, 8, 4]

In [56]:
def idgrab(idfile):
    temp = []
    with open(idfile) as fp:
        for line in fp:
            name = line.split()[1]
            temp.append(name)
    return temp
#print(name)

# This will be used for making random samples of animals who are not on multiple chips
# These animals should all be processed and then merged into one file, they can be pulled out of there. 
# duplicate animals can be merged into seperate file, or kept in their original files
# animals that are f250 and chip should perhaps be prioritized, but with only their f250 data, or with their full complement of data, but not for the animals being tested.  


# For Holstein Test Data:
* We want to have the following numbers of animals in each set
    - HD (10%) = 197
    - F250 (10%) = 197
    - SNP50 (40%) = 788
    - GGPLD (40%) = 788

In [4]:
!head raw_data/777962.170519.1970.HD.ID

1 UMCUSAM000000078648
2 UMCUSAF000000078649
3 UMCUSAM000000078650
4 UMCUSAF000000078651
5 UMCUSAM000000078652
6 UMCUSAM000000078653
7 UMCUSAF000000078654
8 UMCUSAM000000078655
9 UMCUSAF000000078656
10 UMCUSAM000000078657


In [1]:
with open('./raw_genotypes/777962.170519.1970.HD.ID','r') as i:
    idlist = [xx.split()[1] for xx in i]

In [34]:
random.shuffle(idlist)

In [52]:
writeHD = open('./dataprepper/hd_ids.list1.txt','w')
for i in idlist[:197]:
    writeHD.write(' '.join([str(1),i,'\n']))
writeHD.close()

In [53]:
writeF250 = open('./dataprepper/f250_ids.list1.txt','w')
for i in idlist[197:394]:
    writeF250.write(' '.join([str(1),i,'\n']))
writeF250.close()

In [54]:
writeSNP50 = open('./dataprepper/snp50_ids.list1.txt','w')
for i in idlist[394:1182]:
    writeSNP50.write(' '.join([str(1),i,'\n']))
writeSNP50.close()

In [55]:
writeGGPLD = open('./dataprepper/ggpld_ids.list1.txt','w')
for i in idlist[1182:]:
    writeGGPLD.write(' '.join([str(1),i,'\n']))
writeGGPLD.close()

In [43]:
! wc -l ./testset_assays/*list1*.txt

wc: ./testset_assays/*list1*.txt: No such file or directory


In [3]:
def makelists(listnum,seed,High):
    with open('./raw_genotypes/777962.170519.1970.HD.ID','r') as i:
        idlist = [xx.split()[1] for xx in i]
    random.seed(seed)
    random.shuffle(idlist)
    print(idlist[:10])
    writeHD = open('./dataprepper/hd_ids.list'+ listnum+ '.txt','w')
    for i in idlist[:High]:
        writeHD.write(' '.join([str(1),i,'\n']))
    writeHD.close()
    writeF250 = open('./dataprepper/f250_ids.list'+ listnum+ '.txt','w')
    f250up = High*2
    for i in idlist[High:f250up]:
        writeF250.write(' '.join([str(1),i,'\n']))
    writeF250.close()
    writeSNP50 = open('./dataprepper/snp50_ids.list'+ listnum+ '.txt','w')
    left = len(idlist)-f250up
    split = int((left/2) + f250up)
    for i in idlist[f250up:split]:
        writeSNP50.write(' '.join([str(1),i,'\n']))
    writeSNP50.close()
    writeGGPLD = open('./dataprepper/ggpld_ids.list'+ listnum+ '.txt','w')
    for i in idlist[split:]:
        writeGGPLD.write(' '.join([str(1),i,'\n']))
    writeGGPLD.close()

the above code takes a list of all 1970 animals with f250 and hd data and randomizes it. 

It then writes out the full randomlist to seperate files and goes in order. It uses random seeding to always get the same list, but different seeds can be used.

For this big hol test, we will take these same animal sets as our truth animals
They will be just have more animals present during phasing, and more animals avaible during imputation (seperately). 

A step is needed to ensure that animal ids don't overlap

In [1]:
makelists('1',51,300)
! wc -l ./dataprepper/*list2*

NameError: name 'makelists' is not defined

In [47]:
makelists('3',51,500)
! wc -l ./dataprepper/*list3*

['UMCUSAF000000087232', 'UMCUSAM000000079877', 'UMCUSAM000000079073', 'UMCUSAF000000087255', 'UMCUSAM000000079502', 'UMCUSAF000000079340', 'UMCUSAM000000079200', 'UMCUSAM000000079524', 'UMCUSAF000000087251', 'UMCUSAF000000078654']
  500 ./dataprepper/f250_ids.list3.txt
  485 ./dataprepper/ggpld_ids.list3.txt
  500 ./dataprepper/hd_ids.list3 .txt
  500 ./dataprepper/hd_ids.list3.txt
  485 ./dataprepper/snp50_ids.list3.txt
 2470 total


In [38]:
394+788

1182

# Creating Variant Lists

In [47]:
with open ("./dataprepper/9913_SNP50_161214.map", 'r') as s:
    ids = [line.split('\t')[1] for line in s]
with open ("./dataprepper/snp50_snps.txt", 'w') as so:
    for i in ids:
        so.write(''.join([i,'\n']))

In [49]:
with open ("./dataprepper/9913_GGPLDv3_161214.map", 'r') as g:
    gids = [line.split('\t')[1] for line in g]
with open ("./dataprepper/ggpld_snps.txt", 'w') as go:
    for i in gids:
        go.write(''.join([i,'\n']))

In [50]:
!head correct_sex/777962.170519.1970.HD.fam

1 UMCUSAM000000078648 HOLUSAM000065801492 0 1 -9
1 UMCUSAF000000078649 HOLUSAM000122440858 0 0 -9
1 UMCUSAM000000078650 0 0 1 -9
1 UMCUSAF000000078651 HOLUSAM000062368589 0 0 -9
1 UMCUSAM000000078652 HOLUSAM000063050142 0 1 -9
1 UMCUSAM000000078653 HOLUSAM000064541632 0 1 -9
1 UMCUSAF000000078654 HOLUSAM000066236225 0 0 -9
1 UMCUSAM000000078655 0 0 1 -9
1 UMCUSAF000000078656 HOLUSAM000129909510 0 0 -9
1 UMCUSAM000000078657 HOLUSAM000133095623 0 1 -9
